In [3]:
import os
import numpy as np

import math
from collections import Counter

def komsu_oylama(filepath, k=5, radius=None, alpha=0.75, weight=True):
    """
    • radius=None  → k-en yakın; radius=px  → yarıçap içinde
    • weight=True  → 1/dist ağırlıklı oy; False → düz sayım
    • alpha        → baskın sınıf oran eşiği (0–1). Karşılanmazsa kutu değişmez
    """
    import math
    from collections import defaultdict, Counter

    with open(filepath) as f:
        lines = [l.strip() for l in f if l.strip()]

    entries, centers = [], []
    for l in lines:
        p   = l.split()
        cls = int(float(p[0]))
        cx, cy, w, h = map(float, p[1:5])
        conf = float(p[5]) if len(p) > 5 else None
        entries.append([cls, (cx, cy, w, h), conf])
        centers.append((cx, cy))

    new_cls = []
    for i, (cls_i, (cx_i, cy_i, w_i, h_i), _) in enumerate(entries):
        if cls_i == 4:                 # class 4'u koru
            new_cls.append(cls_i)
            continue

        # uzaklık listesi
        dists = [(math.hypot(cx_i - cx_j, cy_i - cy_j), j)
                 for j, (cx_j, cy_j) in enumerate(centers) if j != i]
        dists.sort(key=lambda x: x[0])

        # komşu seçim stratejisi
        neighbors = []
        if radius is not None:
            neighbors = [j for d, j in dists if d <= radius]
        else:
            neighbors = [j for _, j in dists[:k]]

        if not neighbors:
            new_cls.append(cls_i)
            continue

        # oy toplama
        scores = defaultdict(float)
        for j in neighbors:
            cls_j = entries[j][0]
            wght = 1.0 / (dists[[idx for idx, jj in enumerate(neighbors) if jj == j][0]][0] + 1e-6) if weight else 1.0
            scores[cls_j] += wght

        # kazanan sınıf
        winner, win_score = max(scores.items(), key=lambda x: x[1])
        total_score = sum(scores.values())

        if win_score / total_score >= alpha:
            new_cls.append(winner)
        else:
            new_cls.append(cls_i)

    # dosyayı güncelle
    with open(filepath, "w") as f:
        for (old_cls, box, conf), cls in zip(entries, new_cls):
            line = f"{cls} {' '.join(f'{x:.6f}' for x in box)}"
            if conf is not None:
                line += f" {conf:.6f}"
            f.write(line + "\n")

def compute_iou(box1, box2):
    x1_min = box1[0] - box1[2] / 2
    y1_min = box1[1] - box1[3] / 2
    x1_max = box1[0] + box1[2] / 2
    y1_max = box1[1] + box1[3] / 2

    x2_min = box2[0] - box2[2] / 2
    y2_min = box2[1] - box2[3] / 2
    x2_max = box2[0] + box2[2] / 2
    y2_max = box2[1] + box2[3] / 2

    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
    area1 = (x1_max - x1_min) * (y1_max - y1_min)
    area2 = (x2_max - x2_min) * (y2_max - y2_min)

    union_area = area1 + area2 - inter_area
    return inter_area / union_area if union_area > 0 else 0

def remove_duplicates_from_file(filepath, iou_threshold=0.9):
    #print(f"İşleniyor: {filepath}")
    with open(filepath, 'r') as f:
        lines = f.readlines()

    boxes = []
    for line in lines:
        parts = line.strip().split()
        cls = int(parts[0])
        box   = list(map(float, parts[1:5]))  
        boxes.append((cls, box, line.strip()))

    keep = []
    used = [False] * len(boxes)
    
    for i in range(len(boxes)):
        if used[i]:
            continue
        cls1, box1, original1 = boxes[i]
        keep.append((cls1, box1))
        used[i] = True          # ← eklendi
        for j in range(i + 1, len(boxes)):
            cls2, box2, original2 = boxes[j]
            if not used[j]:
                iou = compute_iou(box1, box2)
                if iou > iou_threshold:
                    used[j] = True
                    print(f"[{os.path.basename(filepath)}] Silinen: {original2}  (IoU={iou:.2f})")

    with open(filepath, 'w') as f:
        for cls, box in keep:
            f.write(f"{cls} {' '.join(map(str, box))}\n")

def process_labels_folder(labels_folder, iou_threshold=0.7):
    for filename in os.listdir(labels_folder):
        if filename.endswith('.txt') and filename != "classes.txt":
            filepath = os.path.join(labels_folder, filename)

            # 1) önce KOPYA kutuları sil
            remove_duplicates_from_file(filepath, iou_threshold)

            # 2) sonra komşu oylaması
            #komsu_oylama(filepath, k=10, radius=None, alpha=0.85, weight=True)        # <-- DÜZELTİLDİ

            print(f"Tamamlandı: {filename}\n")


if __name__ == "__main__":
    labels_path = r"C:\Users\524ha\Desktop\Resources\data_managment\notebooks\2010\unlabeled_1\labels"  # Bu klasör yolunu gerektiği gibi değiştir
    
    iou_threshold = 0.5     # Buradan ayarlanabilir
    process_labels_folder(labels_path, iou_threshold)


Tamamlandı: 0508_000003.txt

Tamamlandı: 0508_000006.txt

Tamamlandı: 0508_000009.txt

Tamamlandı: 0508_000012.txt

Tamamlandı: 0508_000015.txt

Tamamlandı: 0508_000018.txt

Tamamlandı: 0508_000021.txt

Tamamlandı: 0508_000024.txt

[0508_000027.txt] Silinen: 3 0.329167 0.533333 0.172917 0.314815  (IoU=0.98)
Tamamlandı: 0508_000027.txt

Tamamlandı: 0508_000030.txt

Tamamlandı: 0508_000033.txt

Tamamlandı: 0508_000036.txt

Tamamlandı: 0508_000039.txt

Tamamlandı: 0508_000042.txt

Tamamlandı: 0508_000045.txt

[0508_000048.txt] Silinen: 5 0.203906 0.490741 0.142187 0.296296  (IoU=0.98)
Tamamlandı: 0508_000048.txt

Tamamlandı: 0508_000051.txt

Tamamlandı: 0508_000054.txt

Tamamlandı: 0508_000057.txt

Tamamlandı: 0508_000060.txt

Tamamlandı: 0508_000063.txt

Tamamlandı: 0508_000066.txt

[0508_000069.txt] Silinen: 5 0.436979 0.543519 0.181250 0.325926  (IoU=0.98)
Tamamlandı: 0508_000069.txt

Tamamlandı: 0508_000072.txt

Tamamlandı: 0508_000075.txt

Tamamlandı: 0508_000078.txt

Tamamlandı: 050